In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle

In [2]:
df = pd.read_csv("Churn_Modelling.csv")
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
df.drop(["RowNumber", "CustomerId", "Surname"], axis = 1, inplace=True)
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CreditScore      10000 non-null  int64  
 1   Geography        10000 non-null  object 
 2   Gender           10000 non-null  object 
 3   Age              10000 non-null  int64  
 4   Tenure           10000 non-null  int64  
 5   Balance          10000 non-null  float64
 6   NumOfProducts    10000 non-null  int64  
 7   HasCrCard        10000 non-null  int64  
 8   IsActiveMember   10000 non-null  int64  
 9   EstimatedSalary  10000 non-null  float64
 10  Exited           10000 non-null  int64  
dtypes: float64(2), int64(7), object(2)
memory usage: 859.5+ KB


In [5]:
df.describe()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,650.528800,38.921800,5.012800,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,96.653299,10.487806,2.892174,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,584.000000,32.000000,3.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,652.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000
max,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000


In [6]:
df.isna().sum()

CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

In [7]:
df['Geography'].unique()

array(['France', 'Spain', 'Germany'], dtype=object)

In [8]:
df['Gender'].unique()

array(['Female', 'Male'], dtype=object)

In [9]:
df['Exited'].value_counts()

Exited
0    7963
1    2037
Name: count, dtype: int64

In [10]:
## Spliting the dataset
X = df.drop("Exited", axis = 1)
y = df["Exited"]

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)

In [12]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [13]:
df.columns

Index(['CreditScore', 'Geography', 'Gender', 'Age', 'Tenure', 'Balance',
       'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary',
       'Exited'],
      dtype='object')

In [14]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer

numeric_cols = ['CreditScore', 'Age', 'Tenure', 'Balance',
                'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary']
onehot_column = ["Geography"]
label_column = ['Gender']

# Numeric Pipeline
num_pipeline = Pipeline([
    ('scaler', StandardScaler())
])

# Label (Ordinal) Pipeline
label_pipeline = Pipeline([
    ('ordinal_encoder', OrdinalEncoder()),
    ('scaler', StandardScaler())
])

# OneHot Encoding Pipeline
oh_pipeline = Pipeline([
    ('oh_encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=False)),
    ('scaler', StandardScaler())
])

# Combined Preprocessor
preprocessor = ColumnTransformer(transformers=[
    ('numeric', num_pipeline, numeric_cols),
    ('label', label_pipeline, label_column),
    ('onehot', oh_pipeline, onehot_column)
], remainder='passthrough')

# Transforming Data
X_train_preprocessed = preprocessor.fit_transform(X_train)
X_test_preprocessed = preprocessor.transform(X_test)


In [15]:
X_train_preprocessed.shape

(8000, 12)

In [16]:
X_test_preprocessed.shape

(2000, 12)

In [17]:
y_train.shape

(8000,)

In [18]:
preprocessor

,transformers,"[('numeric', ...), ('label', ...), ...]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True
,force_int_remainder_cols,'deprecated'
,copy,True
,with_mean,True
,with_std,True


In [19]:
with open("preprocessor_model.pkl", "wb") as file:
    pickle.dump(preprocessor, file)

# ANN IMPLEMENTATION

In [20]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

In [21]:
model = Sequential([
    Dense(units = 64, activation="relu", input_dim = X_train_preprocessed.shape[1]),
    Dense(units = 32, activation="relu"),
    Dense(units = 1, activation="sigmoid")
])

In [22]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [26]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
adam_opt = Adam(learning_rate = 0.01)

In [27]:
model.compile(optimizer=adam_opt, loss="binary_crossentropy", metrics=["accuracy"])

In [28]:
## Setting up the 
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
log__dir = "logs\\fit" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback = TensorBoard(log__dir, histogram_freq=1)

In [29]:
## Setting up early stopping
early_stopping_callback = EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True)

In [36]:
## MOdel training
history = model.fit(
    X_train_preprocessed, y_train, validation_data = (X_train_preprocessed, y_train), epochs = 100,
    callbacks = [tensorflow_callback, early_stopping_callback]
)

Epoch 1/100
250/250 [==============================] - 1s 3ms/step - loss: 0.2109 - accuracy: 0.9056 - val_loss: 0.1884 - val_accuracy: 0.9114
Epoch 2/100
250/250 [==============================] - 1s 2ms/step - loss: 0.2124 - accuracy: 0.9024 - val_loss: 0.1998 - val_accuracy: 0.9074
Epoch 3/100
250/250 [==============================] - 1s 2ms/step - loss: 0.2098 - accuracy: 0.9065 - val_loss: 0.2096 - val_accuracy: 0.9011
Epoch 4/100
250/250 [==============================] - 1s 2ms/step - loss: 0.2142 - accuracy: 0.9007 - val_loss: 0.2068 - val_accuracy: 0.9055
Epoch 5/100
250/250 [==============================] - 1s 3ms/step - loss: 0.2164 - accuracy: 0.9007 - val_loss: 0.1901 - val_accuracy: 0.9118
Epoch 6/100
250/250 [==============================] - 1s 3ms/step - loss: 0.2061 - accuracy: 0.9055 - val_loss: 0.1877 - val_accuracy: 0.9133
Epoch 7/100
250/250 [==============================] - 1s 3ms/step - loss: 0.2151 - accuracy: 0.9015 - val_loss: 0.2017 - val_accuracy: 0.9071

In [37]:
model.save("ANN_Model.h5")  ## keras is more compatible with h5

d:\CODDING\Python\Python_Programs\Projects\DL-Project_1\myenv\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [38]:
## Load TensorBoard Extensrion
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [40]:
%tensorboard --logdir logs\\fit20250628-195528

Reusing TensorBoard on port 6008 (pid 21800), started 0:00:17 ago. (Use '!kill 21800' to kill it.)

In [41]:
y_pred_prob = model.predict(X_test_preprocessed)

63/63 [==============================] - 0s 2ms/step


In [43]:
y_pred_prob.shape

(2000, 1)

In [44]:
y_pred = [1 if y > 0.5 else 0 for y in y_pred_prob]

In [47]:
len(y_pred)

2000

In [48]:
from sklearn.metrics import accuracy_score

In [49]:
score = accuracy_score(y_test, y_pred)
score*100

83.0